In [1]:
import random
from copy import deepcopy
import math
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def load(file):
    cities = []
    with open(file, 'r') as f:
        for line in f:
            data = line.split()
            city_num = int(data[0])-1
            x = float(data[1])
            y = float(data[2])
            demand = float(data[3])
            ready_time = float(data[4])
            due_time = float(data[5])
            
            city = {'city_num' : city_num, 'x' : x, 'y' : y, 'demand' : demand, 'ready_time' : ready_time, 'due_time' : due_time}
            cities.append(city)
            
    return cities   

In [3]:
input1 = load('resources/input_1.txt')
input2 = load('resources/input_2.txt')
input3 = load('resources/input_3.txt')

for x in input1:
    print(x)

{'city_num': 0, 'x': 35.0, 'y': 35.0, 'demand': 0.0, 'ready_time': 0.0, 'due_time': 230.0}
{'city_num': 1, 'x': 41.0, 'y': 49.0, 'demand': 10.0, 'ready_time': 161.0, 'due_time': 171.0}
{'city_num': 2, 'x': 35.0, 'y': 17.0, 'demand': 7.0, 'ready_time': 50.0, 'due_time': 60.0}
{'city_num': 3, 'x': 55.0, 'y': 45.0, 'demand': 13.0, 'ready_time': 116.0, 'due_time': 126.0}
{'city_num': 4, 'x': 55.0, 'y': 20.0, 'demand': 19.0, 'ready_time': 149.0, 'due_time': 159.0}
{'city_num': 5, 'x': 15.0, 'y': 30.0, 'demand': 26.0, 'ready_time': 34.0, 'due_time': 44.0}
{'city_num': 6, 'x': 25.0, 'y': 30.0, 'demand': 3.0, 'ready_time': 99.0, 'due_time': 109.0}
{'city_num': 7, 'x': 20.0, 'y': 50.0, 'demand': 5.0, 'ready_time': 81.0, 'due_time': 91.0}
{'city_num': 8, 'x': 10.0, 'y': 43.0, 'demand': 9.0, 'ready_time': 95.0, 'due_time': 105.0}
{'city_num': 9, 'x': 55.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 97.0, 'due_time': 107.0}
{'city_num': 10, 'x': 30.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 124.0, 

In [22]:
class Individual:
    def __init__(self, data, capacity, num_of_vehicles, num_of_cities, service_time):
        self.num_of_vehicles = num_of_vehicles
        self.num_of_cities = num_of_cities
        self.depot = data[0]
        self.data = data
        self.distance_between_cities = self.calc_all_distances()
        self.capacity = capacity
        self.service_time = service_time
        self.alpha = 2
        self.beta = 5
        self.solution = self.initial_solution_nearest_neighbors()
        self.fitness = self.calc_fitness()

    def __len__(self) -> int:
        return len(self.solution)
    
    def calc_distance(self, city1, city2) -> float:
        distance = math.sqrt((city1["x"] - city2["x"])**2 + (city1["y"] - city2["y"])**2)
        return round(distance, 3)
    
    def calc_all_distances(self) -> [[float]]:
        all_distances = [[0 for _ in range(len(self.data))] for _ in range(len(self.data))]
        
        for i in range(len(self.data)):
            for j in range(len(self.data)):
                if i != j:
                    distance = self.calc_distance(self.data[i], self.data[j])
                    all_distances[i][j] = distance
                else:
                    all_distances[i][j] = 0
                
        return all_distances


    def initial_solution_nearest_neighbors(self):
        routes = [[] for _ in range(self.num_of_vehicles)]
        remaining_cities = [i for i in range(1, self.num_of_cities + 1)]
        
        return self.generate_solution_nearest_neighbors(routes, remaining_cities)
    
    def get_feasable_cities(self, remaining_cities, start_city, current_time, capacity):
            feasable_cities = [(next_city, round(max(self.distance_between_cities[start_city][next_city], 
                                       self.data[next_city]["ready_time"] - current_time - self.service_time), 3)) 
                               for next_city in remaining_cities \
                               if round(current_time + self.distance_between_cities[start_city][next_city] + \
                                        self.service_time, 3) <= 
                                  self.data[next_city]["due_time"] and capacity >= self.data[next_city]["demand"]]
            
            feasable_cities.sort(key = lambda x: x[1])
            return feasable_cities
    
    
    def generate_feasable_routes(self, routes, remaining_cities) -> [[int]]: 
        for route in routes:
            while True:
                if not route:
                    feasable_cities = self.get_feasable_cities(remaining_cities, 0, 0, self.capacity)
                else:
                    _, current_time, remaining_capacity = self.route_fitness(route)
                    feasable_cities = self.get_feasable_cities(remaining_cities, route[-1], current_time, remaining_capacity)

                if not feasable_cities:
                    break
                    
                if random.random() < 0.25:
                    city_index = random.choice(feasable_cities)[0]
                else:
                    city_index = feasable_cities[0][0]

                remaining_cities.remove(city_index)
                route.append(city_index)

                is_unfeasable, _ = self.is_route_unfeasable(route)
                if is_unfeasable:
                    route.pop(-1)
                    remaining_cities.append(city_index)
                    continue
    
        if not remaining_cities:
            return routes
        
        # create an unfeasable route
        for route in routes:
            if not route:
                for city in remaining_cities:
#                     route.append(city)
                    route.append(city)
                    fitness, current_time, remaining_capacity = self.route_fitness(route)
                    is_unfeasable, _ = self.is_route_unfeasable(route)
                    print('route:', route)
                    #print('fitness:', fitness)
                    #print('current_time:', current_time)
                    #print('remaining_capacity:', remaining_capacity)
                    print('is unfeasable:', is_unfeasable)
                    print('------------------------\n----------------------------')
                    route.pop(-1)
                
                
                break
        
        return routes
        
    def generate_solution_nearest_neighbors(self, routes, remaining_cities) -> [int]:        
        generated_routes = self.generate_feasable_routes(routes, remaining_cities)
        return self.create_solution(generated_routes)

    def create_solution(self, routes) -> [int]:
        solution = []
        for i, route in enumerate(routes):
            for city in route:
                solution.append(city)
            solution.append(self.num_of_cities + i + 1)
        
        # remove empty routes
        for city in reversed(solution):
            if city > 100:
                solution.pop(-1)
            else:
                break
                
        return solution


    def generate_solution_sweep(self):
        pass

    def is_route_unfeasable(self, route) -> (bool, int):
        if not route:
            return False, None
        
        current_time = 0
        previous_city = 0
        remaining_capacity = self.capacity 
        
        for current_city in route:
            current_city_data = self.data[current_city]
            distance = self.distance_between_cities[previous_city][current_city]
            
            if current_time + distance + self.service_time > current_city_data["due_time"]:
                print('time violation')
                print('current time + distance + service time =:', current_time + distance + self.service_time)
                print('city due time =', current_city_data["due_time"])
                return True, current_city 
            
            elif remaining_capacity < current_city_data["demand"]:
                print('capacity violation')
                return True, current_city
                
            current_time += distance + self.service_time
            previous_city = current_city
            remaining_capacity -= current_city_data["demand"]
            
        # return to depot
        if current_time + self.distance_between_cities[previous_city][0] + self.service_time > self.depot["due_time"]:
            print('time violation - return to depot')
            return True, previous_city
        
        return False, None
    
    def route_fitness(self, route) -> (float, float, float):
        if not route:
            return 0, 0, self.capacity
        
        fitness = 0
        current_time = 0
        previous_city = 0
        remaining_capacity = self.capacity
        
        for current_city in route:
            current_city_data = self.data[current_city]
            distance = self.distance_between_cities[previous_city][current_city]
            
            if current_time + distance + self.service_time > current_city_data["due_time"]:
                fitness += (current_time + distance + self.service_time - current_city_data["due_time"])*self.alpha
                
            if remaining_capacity < current_city_data["demand"]:
                fitness += (current_city_data["demand"] - remaining_capacity)*self.beta
                
            current_time += distance + self.service_time
            previous_city = current_city
            remaining_capacity -= current_city_data["demand"]
            
        fitness += current_time + self.distance_between_cities[previous_city][0] + self.service_time
        fitness = round(fitness, 3)
        
        return fitness, current_time, remaining_capacity
    
    def get_routes(self) -> [[int]]:
        routes = []
        route = []
        # all routes except for the last route
        for city_index in self.solution:
            if city_index <= self.num_of_cities:
                route.append(city_index)
            else:
                if len(route) > 0:
                    routes.append(route)
                route = []

        # last route
        if len(route) > 0:
            routes.append(route)

        return routes
    
    def calc_fitness(self) -> float:
        routes = self.get_routes()
        
        # more routes than available vehicles
        if sum(1 for r in routes if len(r) > 0) > self.num_of_vehicles:
            return float('inf')

        fitness = 0
        for route in routes:
            if len(route) == 0:
                continue
            f, _, _ = self.route_fitness(route)
            fitness += f
            
        return fitness

    def is_feasable(self) -> bool:
        routes = self.get_routes()
        if sum(1 for r in routes if len(r) > 0) > self.num_of_vehicles:
            print("more vehicles than available violation")
            return False

        for route in routes:
            is_unfeasable, _ = self.is_route_unfeasable(route)
            if is_unfeasable:
                return False
        
        return True

In [23]:
x = Individual(input1, 200, 50, 100, 10)
print(x.solution)
print(x.is_feasable())

y = Individual(input1, 200, 50, 100, 10)
print(y.solution)
print(y.is_feasable())

time violation
current time + distance + service time =: 42.016
city due time = 42.0
route: [14]
is unfeasable: True
------------------------
----------------------------
time violation
current time + distance + service time =: 51.4
city due time = 51.0
route: [36]
is unfeasable: True
------------------------
----------------------------
time violation
current time + distance + service time =: 44.928
city due time = 44.0
route: [63]
is unfeasable: True
------------------------
----------------------------
time violation
current time + distance + service time =: 28.385
city due time = 28.0
route: [92]
is unfeasable: True
------------------------
----------------------------
[59, 95, 98, 61, 85, 84, 48, 60, 93, 101, 5, 24, 3, 50, 1, 77, 80, 102, 72, 2, 37, 17, 96, 91, 100, 58, 103, 42, 35, 68, 25, 104, 27, 28, 69, 88, 7, 46, 13, 105, 33, 30, 79, 55, 54, 70, 106, 45, 82, 19, 49, 32, 107, 39, 23, 67, 56, 4, 74, 108, 21, 15, 43, 57, 26, 89, 109, 83, 86, 16, 94, 97, 110, 10, 76, 81, 34, 111,

In [44]:
x.solution == y.solution

False

## Problem: Nedostižni gradovi

Gradovi 14, 36, 63, 92 predstavljaju nedopustiva rešenja.
Nemoguće ih je opslužiti u zadatom vremenskom roku ukoliko se krene od depoa, što je ujedno i početni trenutak svake rute. 

Kako za svaki grad dobijamo vremensko prekoračenje od početne tačke bilo koje rute, zaključujemo da su ti gradovi u potpunosti nedostižni i nije ih moguće obraditi u zadatom vremenskom roku.

## Rešenje: Postoji više mogućnosti

 - Prvo i intuitivno rešenje koje se nameće je da u potpunosti odbacimo navedene gradove i na dalje da posmatramo samo ostale, dostižne gradove koji proizvode dopustiva rešenja.
 
 - Drugo rešenje bi bilo da ne posmatramo prekoračenje vremenskog roka za svaki od gradova kao nedopustivo rešenje, nego da ga penalizujemo za udeo prekoračenog vremena. Na dalje ćemo ovakve rute smatrati dopustivim.

# Selection

In [ ]:
def random_selection(population):
    return random.choice(population)

In [ ]:
def tournament_selection(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return max(tournament, key = lambda x: x.fitness)

In [ ]:
def roulette_selection(population):
    total_fitness = sum([individual.fitness for individual in population])
    selection_probs = [individual.fitness / total_fitness for individual in population]
    return random.choices(population, weights=selection_probs)[0]

In [ ]:
def rang_selection(population):
    n = len(population)
    rang_sum = n * (n+1) / 2
    selection_probs = []
    for rang, _ in enumerate(sorted(population, key=lambda x: x.fitness), 1):
        selection_probs.append(rang / rang_sum)
        
    return random.choices(population, weights=selection_probs)[0]

In [ ]:
def selection(param, population):
    if param["tournament_size"] is not None:
        return param["selection"](population, param["tournament_size"])
    else:
        return param["selection"](population)

In [ ]:
class Example_Individual:
    def __init__(self, fitness, name):
        self.fitness = fitness
        self.name = name

example_population = [Example_Individual(fitness=0.8, name='A'), 
                      Example_Individual(fitness=0.6, name='B'), 
                      Example_Individual(fitness=0.7, name='C'),
                      Example_Individual(fitness=0.8, name='D'), 
                      Example_Individual(fitness=0.9, name='E')]

print(random_selection(example_population).name)
print(tournament_selection(example_population, 3).name)
print(roulette_selection(example_population).name)
print(rang_selection(example_population).name)

# Crossover

In [ ]:
def order_crossover(parent1, parent2, child1, child2):
    def oc(p1, p2, ch):
        lb, ub = sorted(random.sample(range(len(p1) + 1), 2))
        ch[lb:ub] = p1[lb:ub]

        j = 0
        for i in range(len(p2)):
            if j == lb:
                j = ub + 1

            if j >= len(ch):
                break

            if p2[i] not in ch:
                ch[j] = p2[i]
                j = j + 1 

        return ch
    
    
    child1 = oc(parent1, parent2, child1)
    child2 = oc(parent2, parent1, child2)
    
    return child1, child2

In [ ]:
def partially_mapped_crossover(parent1, parent2, child1, child2):
    cutoff_1, cutoff_2 = sorted(random.sample(range(len(parent1) + 1), 2))
    
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)
    
    child1[cutoff_1:cutoff_2] = parent2[cutoff_1:cutoff_2]
    child2[cutoff_1:cutoff_2] = parent1[cutoff_1:cutoff_2]

   

    middle_map = {}
    for a, b in zip (child1[cutoff_1:cutoff_2], child2[cutoff_1:cutoff_2]):
        middle_map[a] = middle_map.get(a, 0) + 1
        middle_map[b] = middle_map.get(b, 0) + 1
        
    mapping_order = []
    for key, value in middle_map.items():
        if value != 2:
            mapping_order.append(key)
            
    def find_all_occurrences(lst, item):
        return [i for i, x in enumerate(lst) if x == item]    
          
    def replace_a_b(x_pos, y, child, cutoff_1, cutoff_2):
        for i in x_pos:
            if i not in range(cutoff_1, cutoff_2):
                child[i] = y
                break 
                   
        return child
        
    def replace_a_b_child(a, a_pos, b, b_pos, child, cutoff_1, cutoff_2):
        if len(a_pos) != 0:
             return replace_a_b(a_pos, b, child, cutoff_1, cutoff_2)
        else:
            return replace_a_b(b_pos, a, child, cutoff_1, cutoff_2)
        
    for i in range(0, len(mapping_order)-1, 2):
        a = mapping_order[i]
        b = mapping_order[i+1]

        a_child1_positions = find_all_occurrences(child1, a)
        a_child2_positions = find_all_occurrences(child2, a)
        
        
        b_child1_positions = find_all_occurrences(child1, b)
        b_child2_positions = find_all_occurrences(child2, b)
        
        child1 = replace_a_b_child(a, a_child1_positions, b, b_child1_positions, child1, cutoff_1, cutoff_2)
        child2 = replace_a_b_child(a, a_child2_positions, b, b_child2_positions, child2, cutoff_1, cutoff_2)

    return child1, child2

[1 | 2 3 4 | 5]  
[5 | 4 3 1 | 2]   

[1 | 4 3 1 | 5]   
[5 | 2 3 4 | 2]   

4-2, 3-3, 1-4

1: 1   
2: 1   
3: 2   
4: 2  
1 -> 2  

[2 | 1 3 4 | 5]    
[5 | 2 3 4 | 1]    

In [ ]:
def best_route_better_adjustment_crossover(individual, parent1, parent2, child1, child2):
    # n/2 best from parent1 into first n/2 of child1
    # the rest elements are from parent2 
    # the positions of vehicles are fixed
        
    def create_child(p1, p2, ch1):
        p1_routes = individual.get_routes()
        p1_routes.sort(key = lambda route: individual.route_fitness(route)[0])

        # copy half of the best routes from parent1 to child 1
        best_routes_index = 0
        i = 0
        node_index = 0
        while best_routes_index <= len(p1_routes) / 2:
            node = ch1[node_index]
            if node <= individual.num_of_cities and i < len(p1_routes[best_routes_index]):
                ch1[node_index] = p1_routes[best_routes_index][i]
                i = i + 1
            else:
                best_routes_index = best_routes_index + 1
                i = 0

            node_index = node_index + 1

        # copy elements from parent2 that aren't in child so far
        p2_index = 0
        while node_index < len(ch1):
            node = ch1[node_index]
            if node <= individual.num_of_cities:
                p2_node = p2[p2_index]
                if p2_node > individual.num_of_cities:
                    p2_index = p2_index + 1
                    p2_node = p2[p2_index]
                
                if p2_node not in ch1[:node_index]:
                    ch1[node_index] = p2_node
                p2_index = p2_index + 1

            node_index = node_index + 1
            
        return ch1
    
    child1 = create_child(parent1, parent2, child1)
    child2 = create_child(parent2, parent1, child2)
    return child1, child2

In [ ]:
def crossover(param, parent1, parent2, child1, child2):
    if param["individual"] is not None:
        return param["crossover"](param["individual"], parent1, parent2, child1, child2)
    else:
        return param["crossover"](parent1, parent2, child1, child2)

# Mutation

In [ ]:
def swap_mutation(individual):
    l, r = sorted(random.sample(range(len(individual)), 2))
    individual[l], individual[r] = individual[r], individual[l]
    
    return individual

In [ ]:
def invert_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    chosen.reverse()
    individual[l:r] = chosen
    
    return individual

In [ ]:
def shaking_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    random.shuffle(chosen)
    individual[l:r] = chosen
    
    return individual

In [ ]:
def mutation(param, individual, mutation_prob):
    if random.random() < mutation_prob:
        individual = param(individual)
        
    return individual

# Fix solution



    "An adaptive large neighborhood search heuristic for the vehicle routing problem with time windows" by C. R. Castro, M. G. C. Resende, and A. A. Viana. (2012)
        This paper presents an adaptive large neighborhood search (ALNS) heuristic for the VRPTW, which includes an insertion-based repair mechanism.

    "A hybrid genetic algorithm for the vehicle routing problem with time windows" by Y. Liang and J. Zhang. (2006)
        The authors propose a hybrid genetic algorithm that integrates an insertion-based repair method for the VRPTW.

    "An adaptive memory algorithm for a class of vehicle routing problems with time windows" by M. Gendreau, G. Laporte, and F. Semet. (2006)
        This paper introduces an adaptive memory algorithm for vehicle routing problems, including the VRPTW, which utilizes insertion-based repair techniques.

    "An adaptive algorithm for the vehicle routing problem with time windows" by M. Gendreau, G. Laporte, and F. Semet. (2002)
        The authors propose an adaptive algorithm for the VRPTW that employs insertion-based repair strategies to improve solution quality.

    "An adaptive large neighborhood search heuristic for the vehicle routing problem with time windows" by T. G. Crainic, M. Gendreau, and J. Potvin. (2000)
        This paper introduces an adaptive large neighborhood search heuristic for the VRPTW, which includes insertion-based repair procedures to handle infeasible solutions.

ideja: citye koji prave unfeasable rute treba izvuci i staviti u removelistu i onda ih rasporedjujemo tamo gde mozemo, ako ih je nemoguce rasporediti pravimo novu rutu od njih 
ili cvorove u unfeasable rutama preuredimo da budu feasible 


In [99]:
def insertion_based_repair(individual):
    if (individual.is_feasable()):
        print('FESABLE')
        return individual
        
    routes = individual.get_routes()
    unfeasable_routes = []
    for route in routes:
        is_unfeasable, _ = individual.is_route_unfeasable(route)
        if is_unfeasable:
            unfeasable_routes.append(route)
            routes.remove(route)
            
    print('-------NUMBER OF UNFEASABLE ROUTES------    ', len(unfeasable_routes))
            
    removed_cities_list = []
    for route in unfeasable_routes:        
        route.sort(key = lambda x: (individual.data[x]["ready_time"], individual.data[x]["due_time"]))
        
        print('CITIES AFTER SORTING THE ROUTE', route)
        
        # find and eliminate unfeasable cities
        while True:
            is_unfeasable, unfeasable_city = individual.is_route_unfeasable(route)
            if not is_unfeasable:
                break
                
            removed_cities_list.append(unfeasable_city)
            route.remove(unfeasable_city)
  
                    
        print('REMAINING CITIES AFTER REMOVING UNFEASABLE CITIES', route)
            
        routes.append(route)
        
    # first try: insert if possible in existing route
    if len(removed_cities_list) > 0:
        print('ENTERING FIRST TRY IN REPAIR...................................')
        print('\tNUMBER OF UNFEASABLE CITIES', len(removed_cities_list))
        
        for city_index in removed_cities_list:
            is_inserted = False
            for route in routes:            
                for i in range(len(route)):
                    route_copy = deepcopy(route)
                    route_copy.insert(i, city_index)
                    
                    is_unfeasable, _ = individual.is_route_unfeasable(route_copy)
                    if not is_unfeasable:
                        print('\tADDED A CITY TO A ROUTE SUCCESSFULLY')
                        route = deepcopy(route_copy)
                        is_inserted = True
                        removed_cities_list.remove(city_index)
                        break
                        
                if is_inserted:
                    break
                    
    # second try: create new routes 
    if len(removed_cities_list) > 0 and individual.num_of_vehicles - len(routes) > 0:
        print('ENTERING SECOND TRY IN REPAIR...................................')
        print('\tNUMBER OF UNFEASABLE CITIES', len(removed_cities_list))
        
        new_routes = [[] for _ in range( individual.num_of_vehicles - len(routes) )]
        generated_routes = individual.generate_feasable_routes(new_routes, removed_cities_list)
        routes = routes + generated_routes
        
    if individual.num_of_vehicles - len(routes) > 0:
        print('\tREMOVED CITIES LIST IS EMPTY, ADD SORTED ROUTES TO INDIVIDUAL............................')
        individual.solution = individual.create_solution(routes)
        
    
    return individual

In [100]:
x = Individual(input1, 200, 50, 100, 10)
y = Individual(input1, 200, 50, 100, 10)
x_child = Individual(input1, 200, 50, 100, 10)
y_child = Individual(input1, 200, 50, 100, 10)

def print_solution_and_feasibility(x, y):
    print('\t', 'x solution:', x.solution)
    print('\t', 'is x feasable:', x.is_feasable())
    print('\t', 'y solution:', y.solution)
    print('\t', 'is y feasable:', y.is_feasable())
    print('\n########################################################\n')

print('initial individuals..........')
print_solution_and_feasibility(x, y)
x_solution_previous = x.solution
y_solution_previous = y.solution

print('order crossover.........')
x_child.solution, y_child.solution = order_crossover(x.solution, y.solution, x_child.solution, y_child.solution)
print('\t', 'same x solutions:', x_child.solution == x_solution_previous)
print('\t', 'same y solutions:', y_child.solution == y_solution_previous)
print_solution_and_feasibility(x_child, y_child)
x_solution_previous = x_child.solution
y_solution_previous = y_child.solution

print('insertion based repair........')
print('x child')
x_child = insertion_based_repair(x_child)
print('y child')
y_child = insertion_based_repair(y_child)
print('\t', 'same x solutions:', x_child.solution == x_solution_previous)
print('\t', 'same y solutions:', y_child.solution == y_solution_previous)
print_solution_and_feasibility(x_child, y_child)

initial individuals..........
	 x solution: [93, 95, 56, 22, 43, 91, 100, 58, 101, 46, 19, 90, 10, 32, 70, 102, 59, 83, 60, 99, 94, 26, 4, 25, 103, 5, 82, 88, 74, 13, 89, 104, 72, 78, 79, 3, 1, 48, 105, 42, 2, 73, 68, 50, 77, 80, 106, 27, 28, 37, 85, 84, 97, 17, 107, 33, 81, 29, 9, 34, 24, 108, 45, 44, 86, 96, 109, 6, 69, 76, 53, 54, 110, 39, 23, 67, 55, 111, 31, 52, 30, 51, 20, 35, 112, 47, 11, 8, 113, 62, 64, 49, 114, 98, 15, 87, 57, 115, 65, 71, 66, 116, 21, 12, 18, 117, 75, 40, 41, 118, 16, 7, 119, 61, 38]
	 is x feasable: True
	 y solution: [55, 12, 40, 50, 48, 101, 59, 99, 98, 16, 85, 84, 96, 13, 89, 93, 102, 5, 83, 61, 86, 43, 91, 100, 58, 103, 72, 41, 23, 22, 56, 54, 24, 80, 104, 29, 76, 70, 10, 32, 1, 77, 105, 82, 47, 19, 90, 20, 35, 106, 42, 53, 66, 107, 27, 28, 69, 3, 79, 34, 68, 25, 108, 33, 30, 51, 81, 26, 4, 109, 9, 71, 78, 74, 110, 95, 49, 46, 17, 60, 111, 45, 44, 87, 97, 37, 112, 39, 73, 57, 113, 2, 38, 114, 31, 52, 88, 7, 8, 115, 65, 116, 62, 11, 64, 117, 15, 75, 94, 1

AttributeError: 'Individual' object has no attribute 'generate_feasable_routes'

# Genetic algorithm

In [ ]:
def genetic_algorithm(params):
    
    data = params["data"]
    capacity = params["capacity"]
    num_of_vehicles = params["num_of_vehicles"]
    service_time = params["service_time"]
    population_size = params["population_size"]
    num_generations = params["num_generations"]
    elitism_size = params["elitism_size"]
    tournament_size = params["tournament_size"]
    selection_params = params["selection"]
    crossover_param = params["crossover"]
    mutation_params = params["mutation"]
    mutation_prob = params["mutation_prob"]
    
    population = [Individual(data, capacity, num_of_vehicles, len(data)-1, service_time ) for _ in range(population_size)]
    print([x.fitness for x in population])
    new_population = deepcopy(population)
  
    crossover_params = { "crossover" : crossover_param, 
                         "individual" : population[0] if crossover_param.__name__ == "best_route_better_adjustment_crossover" else None }
    best_solutions = []
    for i in range(num_generations):
        population.sort(key = lambda x: x.fitness)
        print([x.fitness for x in population])
        best_solutions.append(population[0])
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = selection(selection_params, population)
            parent2 = selection(selection_params, population)
            
            while(parent1 == parent2):
                parent2 = selection(selection_params, population)

            
            new_population[j].solution, new_population[j+1].solution = crossover(crossover_params, 
                                                                                 parent1.solution, 
                                                                                 parent2.solution, 
                                                                                 new_population[j].solution, 
                                                                                 new_population[j+1].solution)
                
            
            new_population[j].solution = mutation(mutation_params, new_population[j].solution, mutation_prob)
            new_population[j+1].solution = mutation(mutation_params, new_population[j+1].solution, mutation_prob)

            insertion_based_repair(new_population[j])
            insertion_based_repair(new_population[j+1])

            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()

        population = deepcopy(new_population)
        
    
    return min(population, key = lambda x: x.fitness), best_solutions

In [ ]:
POPULATION_SIZE = 1000
ELITISIM_SIZE = 100
MUTATION_PROB = 0.1
TOURNAMENT_SIZE = 300
NUM_GENERATIONS = 100
CAPACITY = 300
SELECTION = rang_selection
CROSSOVER = order_crossover
MUTATION = shaking_mutation
NUM_OF_VEHICLES = 50
SERVICE_TIME = 10

params = {
    "data" : input1,
    "population_size" : POPULATION_SIZE,
    "elitism_size" : ELITISIM_SIZE,
    "mutation_prob" : MUTATION_PROB,
    "tournament_size" : TOURNAMENT_SIZE,
    "num_generations" : NUM_GENERATIONS,
    "capacity" : CAPACITY,
    "selection" : { "selection" : SELECTION, 
                    "tournament_size" : TOURNAMENT_SIZE if SELECTION.__name__ == "tournament_selection" else None
                  },
    "crossover" : CROSSOVER,
    "mutation" : MUTATION,
    "num_of_vehicles" : NUM_OF_VEHICLES,
    "service_time" : SERVICE_TIME
}

ga_order_crossover, best_solutions_order_crossover = genetic_algorithm(params)

# params["crossover"] = partially_mapped_crossover

# ga_pmc, best_solutions_pmc = genetic_algorithm(params)

# params["crossover"] = best_route_better_adjustment_crossover

# ga_brbac, best_solutions_brbac = genetic_algorithm(params)


In [ ]:
ga_order_crossover.is_feasable()

In [ ]:
print(sum(1 for route in ga_order_crossover.get_routes() if len(route) > 0))
print(len(ga_order_crossover.get_routes()))
print(ga_order_crossover.fitness)

In [ ]:
ga_pmc.is_feasable()

In [ ]:
ga_brbac.is_feasable()

In [ ]:
ga_brbac.solution == ga_pmc.solution

egzaktan, grafovi, viseciljna, reference, prezentacija i pdf



# Graphs

In [ ]:
def plot_cities(x_coordinates, y_coordinates, dataset_name):
    plt.figure(figsize=(8, 6))
    plt.scatter(x_coordinates[1:], y_coordinates[1:], color='red', label='cities')
    plt.scatter(x_coordinates[0], y_coordinates[0], color='blue', label='Depot') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('City Coordinates for ' + dataset_name)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
x_input1 = [x["x"] for x in input1]
y_input1 = [x["y"] for x in input1]

x_input2 = [x["x"] for x in input2]
y_input2 = [x["y"] for x in input2]

x_input3 = [x["x"] for x in input3]
y_input3 = [x["y"] for x in input3]


plot_cities(x_input1, y_input1, "input_1")


In [ ]:
#Best solution routes 
def plot_best_solution(x, title):
    plt.figure(figsize=(8, 6))
    routes = x.get_routes()
    for route in routes:
        x_points  = np.array([x.depot["x"]])
        x_points = np.append(x_points, [x.data[i]["x"] for i in route], axis = 0)
        x_points = np.append(x_points, [x.depot["x"]], axis = 0)
        y_points  = np.array([x.depot["y"]])
        y_points = np.append(y_points, [x.data[i]["y"] for i in route], axis = 0)
        y_points = np.append(y_points, [x.depot["y"]], axis = 0)
        

        plt.plot(x_points, y_points)
    
    plt.title(title)
    plt.show


In [ ]:
plot_best_solution(ga_order_crossover, "Best Solution Routes OC")
plot_best_solution(ga_pmc, "Best Solution Routes PMC")
plot_best_solution(ga_brbac, "Best Solution Routes BRBAC") 

In [ ]:
def best_solution_evolution(solutions, title, info):
    plt.figure(figsize=(8, 6))
    y_points = [[] for _ in range(len(solutions))]
    i = 0
    for solution in solutions:
        y_points[i] = [x.fitness for x in solution]
        i+=1
    
    for i,y in enumerate(y_points):
        plt.plot(np.array(y), label = info[i])
    plt.title(title)
    plt.ylabel("fitness")
    plt.xlabel("generation")
    plt.legend()
    plt.show()

In [ ]:
best_solution_evolution([best_solutions_order_crossover,best_solutions_brbac, best_solutions_pmc], "The Effect Of Crossover Function On Fitness Using Nearest Neighbour Initialization", ["Order Crossover", "Best Route Better Adjustment Crossover", "Partially Mapped Crossover"])